In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from matplotlib import ticker as mticker
%matplotlib inline


# 1 Read in necessary data

In [ ]:
# Read in ECF, demographic, and IRA classification data and merge together
totalECF_demo = pd.read_csv(
    'totalECF_demo.csv',
    index_col=0,
    dtype={'FIPS': str, 'FIPSTATE': str}
)

ECF_sector = pd.read_csv(
    'https://raw.githubusercontent.com/kailingraham/GrahamKnittel_EmploymentCarbonFootprints_Data/main/ECF_sector.csv',
    index_col=0,
    dtype={'FIPS': str, 'FIPSTATE': str}
)
ECF_sector = ECF_sector.rename(
    columns={'tonneCO2e_eff_peremp_avg': 'ECF',
             'tonneCO2e_eff_peremp_avg_log10': 'ECF_log10'}
)
ECF_sector['tonneCO2e_eff_avg'] = ECF_sector.tonCO2e_eff_avg * 0.907185

ira_counties_ecf = pd.read_csv(
    'ira_counties_ecf.csv',
    index_col=0,
    dtype={'FIPS': str, 'FIPSTATE': str}
)

totalECF_demo = totalECF_demo.merge(
    ira_counties_ecf[[
        col for col in ira_counties_ecf.columns if col not in totalECF_demo.columns] + ['FIPS']],
    on='FIPS',
    how='left'
)

# 2 Prepare mapping dataframe


In [ ]:
# Load in a GeoJSON file containing the geometry information for US counties, where feature.id is a FIPS code.
from urllib.request import urlopen
import json
import geopandas

with urlopen("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json") as response:
    counties = json.load(response)

# Create geodataframe from the same file
counties_gdf = geopandas.read_file("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json")

In [ ]:
# Filter out necessary columns for mapping dataframe
from shapely.geometry.collection import GeometryCollection
totalECF_demo_tool = totalECF_demo[
    ['FIPS', 'County', 'FIPSTATE', 'State',
     'ECF', 'ECF_log10',
     'Emp', 'POP', 'POP_log10', 'MIG_TOT', 'MIG_TOT_log10',
     'MIG_PERCENT', 'UNEMP_RATE', 'POV_RATE', 'INC_IND_TOT',
     'ED_PERCENT_TERTIARY', 'pop_per_sqmi', 'pop_per_sqmi_log10',
     'county_preferred_party', 'RUCC_2013',
     'county_percent_D', 'county_percent_R', 'ETHN_LATIN', #'RACE_NONWHITE_PERCENT',
     'ETHN_LATIN_PERCENT','RACE_BLACK_PERCENT', 'RACE_NATAMERICAN_PERCENT', 'RACE_ASIAN_PERCENT',
     'RACE_HAWAII_PACISLAND_PERCENT', 'RACE_OTHER_PERCENT', 'RACE_PERCENT_MINORITY',
     'RACE_PERCENT_MINORITY_log10', 'ec', 'ec_ffe', 'ec_coal'
     ]
]
# totalECF_demo_tool['MIG_TOT_log10'] = totalECF_demo_tool.apply(
#     lambda x: 0 if x.MIG_TOT_log10 == -np.infty else x.MIG_TOT_log10, axis=1)
mean, std = totalECF_demo_tool.ECF_log10.mean(), totalECF_demo_tool.ECF_log10.std()
totalECF_demo_tool['ECF_log10_zscore'] = totalECF_demo_tool.ECF_log10.apply(
    lambda x: (x - mean) / std)

totalECF_demo_tool = totalECF_demo_tool.rename(
    columns={'RACE_BLACK_PERCENT': 'Black', 'RACE_NATAMERICAN_PERCENT': 'Native American', 'RACE_ASIAN_PERCENT': 'Asian',
             'RACE_HAWAII_PACISLAND_PERCENT': 'Hawaiian/Pac. Islander', 'RACE_OTHER_PERCENT': 'Other'})

# Determine and calculate pop shares of top and second top races per county
totalECF_demo_tool['top_race'] = totalECF_demo_tool[
    ['Black', 'Native American', 'Asian', 'Hawaiian/Pac. Islander', 'Other']
].idxmax(axis='columns', skipna=True)
totalECF_demo_tool['top_race_percent'] = totalECF_demo_tool.apply(
    lambda x: x[x.top_race] if type(x.top_race) == str else np.nan, axis=1)

next_top_races = []
for i in range(len(totalECF_demo_tool)):
    df = pd.DataFrame(totalECF_demo_tool.loc[
        i, [race for race in ['Black', 'Native American', 'Asian', 'Hawaiian/Pac. Islander', 'Other']
         if race != totalECF_demo_tool.loc[i, 'top_race']]
    ]).T.astype(float)
    next_top_races.append(df.idxmax(axis='columns', skipna=True).values[0])

totalECF_demo_tool['next_top_race'] = next_top_races
totalECF_demo_tool['next_top_race_percent'] = totalECF_demo_tool.apply(
    lambda x: x[x.next_top_race] if type(x.next_top_race) == str else np.nan, axis=1)

# Create geodataframe with totalECF_demo data and county geometries
totalECF_demo_tool = counties_gdf.rename(
    columns={'id': 'FIPS'}
)[['FIPS', 'CENSUSAREA', 'geometry']].merge(totalECF_demo_tool, how='right', on='FIPS')

# Deal with empty geometry fields
totalECF_demo_tool['geometry'] = totalECF_demo_tool.geometry.apply(
    lambda x: x if x else GeometryCollection())

# Save to geojson
totalECF_demo_tool.to_file('totalECF_demo_tool.geojson', driver='GeoJSON')
totalECF_demo_tool.to_file('totalECF_demo_tool.json')

# 3 Prepare socioeconomic dataframe

In [ ]:
# Filter out columns needed for the socioeconomic pop-up dashboard
socioec_ECF = totalECF_demo[
    ['FIPS', 'County', 'FIPSTATE', 'State',
     'tonneCO2e_eff_avg', 'ECF', 'ECF_log10',
     'POP', 'Emp', 'UNEMP_RATE', 'POV_RATE', 'INC_IND_TOT', 'ED_PERCENT_TERTIARY'
     ]
]

# Rename & reformat columns
socioec_ECF['County'] = socioec_ECF['County'].str.capitalize()
socioec_ECF = socioec_ECF.rename(
    columns={
        'County': 'COUNTY',
        'State': 'STATE',
        'tonneCO2e_eff_avg': 'TONNECO2E',
        'Emp': 'EMP_TOT'})

# Pivot sectoral data to give tonneCO2e and emp per sector for each county
socioec_ECF_sectoral = ECF_sector.pivot_table(
    index='FIPS',
    columns='sector',
    values=['Emp', 'tonneCO2e_eff_avg']
).reset_index()
for sector in ECF_sector.sector.unique():
    socioec_ECF_sectoral[f'EMP_{sector}'] = socioec_ECF_sectoral[('Emp', sector)]
    socioec_ECF_sectoral[f'TONNECO2E_{sector}'] = socioec_ECF_sectoral[('tonneCO2e_eff_avg', sector)]
socioec_ECF_sectoral = socioec_ECF_sectoral.drop(
    columns=['Emp', 'tonneCO2e_eff_avg']).droplevel(level=1, axis=1)

# Merge sectoral data onto total county data
socioec_ECF = pd.merge(socioec_ECF, socioec_ECF_sectoral, how='left', on='FIPS')
for sector in ECF_sector.sector.unique():
    socioec_ECF[f'EMP_{sector}'] = socioec_ECF.apply(
        lambda x: x[f'EMP_{sector}'] if x[f'EMP_{sector}'] > 0 else
        (x[f'EMP_{sector}'] if x[f'TONNECO2E_{sector}'] > 0 else 0),
        axis=1
        )
    socioec_ECF[f'TONNECO2E_{sector}'] = socioec_ECF[f'TONNECO2E_{sector}'].fillna(0)
    socioec_ECF[f'EMP_{sector.upper()}_PERCENT'] = socioec_ECF[f'EMP_{sector}'] / \
        socioec_ECF.EMP_TOT * 100
    socioec_ECF[f'TONNECO2E_{sector.upper()}_PERCENT'] = socioec_ECF[f'TONNECO2E_{sector}'] / \
        socioec_ECF.TONNECO2E * 100
    socioec_ECF = socioec_ECF.drop(columns=[f'EMP_{sector}', f'TONNECO2E_{sector}'])

# Create state and national ECF figures
totalECF_state_demo = totalECF_demo[
    ['FIPSTATE', 'State', 'tonneCO2e_eff_avg', 'burden_avg', 'POP', 'Emp']
].groupby(by=['FIPSTATE', 'State'], as_index=False).sum()
totalECF_state_demo['ECF'] = totalECF_state_demo.tonneCO2e_eff_avg / \
    totalECF_state_demo.Emp
totalECF_state_demo['ECF_log10'] = np.log10(
    totalECF_state_demo['ECF']
)
mean = np.mean(totalECF_state_demo.ECF_log10)
std = np.std(totalECF_state_demo.ECF_log10)
totalECF_state_demo = totalECF_state_demo.rename(
    columns={'ECF': 'ECF_ST',
             'ECF_log10': 'ECF_ST_log10'}
)
totalECF_state_demo = totalECF_state_demo[['FIPSTATE', 'ECF_ST', 'ECF_ST_log10']]

totalECF_US = totalECF_demo.tonneCO2e_eff_avg.sum() / totalECF_demo.Emp.sum()
totalECF_US_log10 = np.log10(totalECF_US)

# Merge onto total county data
socioec_ECF = pd.merge(socioec_ECF, totalECF_state_demo, how='left', on='FIPSTATE')
socioec_ECF['ECF_US'] = totalECF_US
socioec_ECF['ECF_US_log10'] = totalECF_US_log10


In [ ]:
# Write to CSV
socioec_ECF.to_csv('socioec_ECF.csv')

# 4 Prepare emissions, employment and ECF dataframes for PanelApp.svelte

## 4.1 Prepare sectoral emissions percentage dataframe

In [ ]:
# Isolate relevant columns and rename
emissions = socioec_ECF[['FIPS', 'TONNECO2E_AG_PERCENT', 'TONNECO2E_CN_PERCENT',
       'TONNECO2E_COMM_PERCENT', 
       'TONNECO2E_MF_PERCENT', 'TONNECO2E_MN_REST_PERCENT', 'TONNECO2E_PWR_PERCENT',
       'TONNECO2E_OG_PERCENT',  'TONNECO2E_COAL_PERCENT', ]]
emissions = emissions.rename(
    columns={
       'TONNECO2E_AG_PERCENT': 'Agriculture',
       'TONNECO2E_CN_PERCENT': 'Construction',
       'TONNECO2E_COMM_PERCENT': 'Commercial',
       'TONNECO2E_MF_PERCENT': 'Manufacturing', 
       'TONNECO2E_MN_REST_PERCENT': 'Other mining',
       'TONNECO2E_PWR_PERCENT': 'Power',
       'TONNECO2E_OG_PERCENT': 'Oil and gas', 
       'TONNECO2E_COAL_PERCENT': 'Coal mining', 
    }
)

# Reindex and reorient
emissions = emissions.T.reset_index()
emissions.columns = emissions.iloc[0]
emissions = emissions.iloc[1:]
emissions = emissions.rename(columns={'FIPS': 'sector'})

# Save new format
emissions.to_csv('emissions.csv')

## 4.2 Prepare sectoral employment percentages dataframe

In [ ]:
# Isolate relevant columns and rename
employment = socioec_ECF[['FIPS', 'EMP_AG_PERCENT', 'EMP_CN_PERCENT',
        'EMP_COMM_PERCENT', 'EMP_MF_PERCENT',
       'EMP_MN_REST_PERCENT', 'EMP_PWR_PERCENT', 'EMP_OG_PERCENT', 'EMP_COAL_PERCENT']]
employment = employment.rename(
    columns={
       'EMP_AG_PERCENT': 'Agriculture',
       'EMP_CN_PERCENT': 'Construction',
       'EMP_COMM_PERCENT': 'Commercial',
       'EMP_MF_PERCENT': 'Manufacturing', 
       'EMP_MN_REST_PERCENT': 'Other mining',
       'EMP_PWR_PERCENT': 'Power',
       'EMP_OG_PERCENT': 'Oil and gas', 
       'EMP_COAL_PERCENT': 'Coal mining', 
    }
)

# Reindex and reorient
employment = employment.T.reset_index()
employment.columns = employment.iloc[0]
employment = employment.iloc[1:]
employment = employment.rename(columns={'FIPS': 'sector'})

# Save new format
employment.to_csv('employment.csv')

## 4.3 Prepare ECF dataframe to summarize data at the county, state and national level

In [ ]:
# Isolate relevant columns and rename
ecf = socioec_ECF[['FIPS', 'ECF', 'ECF_ST', 'ECF_US']]
ecf = ecf.rename(
    columns={
       'ECF': 'County', 'ECF_ST': 'State', 'ECF_US': 'National'
    }
)
ecf = np.round(ecf)

# Reindex and reorient
ecf = ecf.T.reset_index()
ecf.columns = ecf.iloc[0]
ecf = ecf.iloc[1:]
ecf = ecf.rename(columns={'FIPS': 'granularity'})

# Save new format
ecf.to_csv('ecf.csv')